In [ ]:
from astropy.table import Table
from astropy.coordinates import SkyCoord

In [ ]:
results = Table.read('./catalogs/results_table_inspected_flagged_0.5_2.0_v2.fits')

# remove duplicates, because they aren't good sources
results = results[~results['DUPLICATE']]

In [ ]:
results_df = results.to_pandas()

In [ ]:
results_df['RA_sex'] = ''
results_df['DEC_sex'] = ''

for i, row in results_df.iterrows():
    coord = SkyCoord(ra=row.RA_detc, dec=row.DEC_detc, frame='icrs', unit='degree')
    results_df.at[i, 'RA_sex'] = coord.ra.to_string(unit='hour', sep=':')
    results_df.at[i, 'DEC_sex'] = coord.dec.to_string(unit='degree', sep=':')

results_df = results_df.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)

In [ ]:
results_df.columns

In [ ]:
# select 30 random entries from the catalog.
small_results_df = results_df.sample(30).sort_values('NAME')

In [ ]:

# here we are going to print out the table
# this should then be copy-and-pasted into the latex
# i found that works a bit better than trying to use 
# astropy to write the table directly

print(r'Field & $\alpha$ (J2000) & $\delta$ (J2000) & $z$ & Exp Time & S/N & Flux & Luminosity & Flags \\')
print(r' & & & & s & & cnts/s (10^{-13}) & ergs/s (10^{44}) & \\')
print(r'(1) & (2) & (3) & (4) & (5) & (6) & (7) & (8) & (9) \\')
for i, row in small_results_df.iterrows():
    # get all the bits
    name = row.NAME
    ra = row.RA_sex
    dec = row.DEC_sex
    z = row.REDSHIFT
    exptime = row.EXPTIME
    sn = row.SN
    flux = row.FLUX_CORR * 1e13
    flux_err = (row.FLUX_CORR_NEGERR * 1e13, row.FLUX_CORR_POSERR * 1e13)
    lum = row.LUM_CORR / 1e44
    lum_err = (row.LUM_CORR_NEGERR / 1e44, row.LUM_CORR_POSERR / 1e44)
    
    print(f'{name} & {ra[:-2]} & {dec[:-2]} ', end='& ')
    if z > 0:
        print(f'{z:.3f}', end=' & ') 
    else:
        print(f'---', end=' & ')
        
    print(f'{exptime:.1f}', end=' & ')    
        
    if sn > 0:
        print(f'{sn:.2f}', end=' & ')
    else:
        print(f'---', end=' & ')
        
    if flux > 0:
        print(f'{flux:.3f}$\pm{{{flux_err[0]:.3f}}}$', end=' & ')
    else:
        print(f'---', end=' & ')
        
    if lum > 0 and z > 0:
        print(f'{lum:.3f}$\pm{{{lum_err[0]:.3f}}}$', end=' & ')
    else:
        print(f'---', end=' & ')
        
    print(f'{row.FLAGS} \\\\')